In [1]:
import pandas as pd
import unicodedata
from commons import DATASET_LOCATION, DATASET_CLEAN_LOCATION, DATASET_CLEAN_UNDERSAMPLING_LOCATION


In [ ]:
df = pd.read_csv(DATASET_LOCATION)

In [3]:
df_iteng = df[(df['Language'] == 'Italian') | (df['Language'] == 'English')].copy()

In [4]:
# Per prima cosa metto tutto in minuscolo. Poi sostituisco l'apostrofo con uno spazio (Altrimenti costrutti come l'uomo diventavano un unico token luomo)
# Successivamente elimino i numeri e poi i caratteri non alfanumerici a parte gli spazi. Elimino infine gli spazi multipli in modo da avere solo uno spazio che divide le parole


def clean_text_keep_accents(text):
    cleaned = ''.join(
        c for c in text
        if (
            # Tieni lettere o numeri o spazi
            unicodedata.category(c).startswith(('L', 'N')) or c.isspace()
        )
        # Per eliminare i caratteri speciali che sono classificati come MODIFIER LETTER
        and not unicodedata.name(c, '').startswith('MODIFIER')
    )
    return cleaned


def clean_text(text):
    text = text.lower()
    text = text.replace("'", " ")
    text = ''.join(c for c in text if not c.isdigit())
    text = ''.join(c for c in text if c.isalnum() or c.isspace())
    #Pulizia dei caratteri unicode
    text = clean_text_keep_accents(text)
    text = ' '.join(text.split())
    return text

In [5]:
df_iteng['Text_clean'] = df_iteng['Text'].apply(clean_text)

In [6]:
df_clean = df_iteng[["Text_clean", "Language"]]
df_clean = df_clean.rename(columns={"Text_clean": "Text"})

In [7]:
df_clean.to_csv(DATASET_CLEAN_LOCATION, index=False)

In [8]:
## Undersampling utilizzando i quartili
df_iteng['Word_count'] = df_iteng['Text_clean'].apply(lambda x: len(str(x).split()))
english_df = df_iteng[df_iteng['Language'] == 'English'].copy()
english_df['quartile'] = pd.qcut(english_df['Word_count'], q=4, labels=False)

undersampled_parts = []
for q in range(4):
    group = english_df[english_df['quartile'] == q]
    sampled = group.sample(frac=0.5, random_state=1999)
    undersampled_parts.append(sampled)
english_undersampled = pd.concat(undersampled_parts)
italian_df = df_iteng[df_iteng['Language'] == 'Italian']
df_clean_undersampling = pd.concat([english_undersampled, italian_df]).reset_index(drop=True)

In [9]:
df_clean_undersampling = df_clean_undersampling[["Text_clean", "Language"]]
df_clean_undersampling = df_clean_undersampling.rename(columns={"Text_clean": "Text"})

In [10]:
df_clean_undersampling.to_csv(DATASET_CLEAN_UNDERSAMPLING_LOCATION, index=False)